In [2]:
import tensorflow as tf


In [3]:
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam

In [ ]:
class RWMAB(Layer):
    """Residual Whole Map Attention Block"""
    def __init__(self, filters, **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        
        # Feature extraction path
        self.conv1 = layers.Conv2D(filters, 3, padding='same')
        self.relu = layers.ReLU()
        self.conv2 = layers.Conv2D(filters, 3, padding='same')
        
        # Whole map attention path
        self.attention_conv = layers.Conv2D(1, 1, activation='sigmoid')
        
        # Residual connection
        self.multiply = layers.Multiply()
        self.add = layers.Add()

    def call(self, inputs):
        # Feature extraction
        x = self.conv1(inputs)
        x = self.relu(x)
        x = self.conv2(x)
        
        # Generate attention map
        attention = self.attention_conv(inputs)
        
        # Apply attention to processed features
        x = self.multiply([x, attention])
        
        # Add residual connection
        return self.add([x, inputs])

    def get_config(self):
        config = super().get_config()
        config.update({"filters": self.filters})
        return config

In [ ]:
class RWMANBlock(Layer):
    """Group of RWMAB blocks with residual connection"""
    def __init__(self, filters, num_blocks, **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.num_blocks = num_blocks
        self.blocks = [RWMAB(filters) for _ in range(num_blocks)]
        self.add = layers.Add()

    def call(self, inputs):
        x = inputs
        for block in self.blocks:
            x = block(x)
        return self.add([x, inputs])

    def get_config(self):
        config = super().get_config()
        config.update({
            "filters": self.filters,
            "num_blocks": self.num_blocks
        })
        return config
class RWMANBlock(Layer):
    """Group of RWMAB blocks with residual connection"""
    def __init__(self, filters, num_blocks, **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.num_blocks = num_blocks
        self.blocks = [RWMAB(filters) for _ in range(num_blocks)]
        self.add = layers.Add()

    def call(self, inputs):
        x = inputs
        for block in self.blocks:
            x = block(x)
        return self.add([x, inputs])

    def get_config(self):
        config = super().get_config()
        config.update({
            "filters": self.filters,
            "num_blocks": self.num_blocks
        })
        return config


In [ ]:

class PixelShuffle(Layer):
    """Custom layer for pixel shuffle upsampling"""
    def __init__(self, block_size, **kwargs):
        super().__init__(**kwargs)
        self.block_size = block_size

    def call(self, inputs):
        return tf.nn.depth_to_space(inputs, self.block_size)

    def get_config(self):
        config = super().get_config()
        config.update({"block_size": self.block_size})
        return config

In [ ]:

class RWMAN(Model):
    """Residual Whole Map Attention Network"""
    def __init__(self, 
                 input_shape=(None, None, 3),
                 filters=64,
                 num_blocks=128,
                 use_noise_channel=False,
                 **kwargs):
        super().__init__(**kwargs)
        
        # Store configuration
        self.initial_filters = filters
        self.num_blocks = num_blocks
        self.use_noise_channel = use_noise_channel
        self.input_channels = 4 if use_noise_channel else 3
        
        # Define layers
        self.input_layer = layers.Input(shape=input_shape[:2] + (self.input_channels,))
        self.initial_conv = layers.Conv2D(filters, 3, padding='same')
        
        # RWMAB groups
        self.num_groups = num_blocks // 16
        self.rwman_blocks = [
            RWMANBlock(filters, 16) 
            for _ in range(self.num_groups)
        ]
        
        # Upsampling layers
        self.upsample_layers = [
            self._create_upsample_block(filters * 4)
            for _ in range(2)  # 2x upsampling blocks
        ]
        
        # Final output layer
        self.final_conv = layers.Conv2D(3, 3, padding='same', activation='tanh')
        
        # Build model
        self._build_model()

    def _create_upsample_block(self, filters):
        return [
            layers.Conv2D(filters, 3, padding='same'),
            PixelShuffle(2),
            layers.PReLU(shared_axes=[1, 2])
        ]

    def _build_model(self):
        # Initial processing
        x = self.initial_conv(self.input_layer)
        long_residual = x
        
        # Process through RWMAN blocks
        for block in self.rwman_blocks:
            x = block(x)
        
        # Long residual connection
        x = layers.Add()([x, long_residual])
        
        # Upsampling
        for upsample_block in self.upsample_layers:
            for layer in upsample_block:
                x = layer(x)
        
        # Final output
        outputs = self.final_conv(x)
        
        # Create model
        self.model = Model(inputs=self.input_layer, outputs=outputs, name=self.name)

    def call(self, inputs):
        return self.model(inputs)

    def get_config(self):
        config = super().get_config()
        config.update({
            "input_shape": self.input_layer.shape[1:],
            "filters": self.initial_filters,
            "num_blocks": self.num_blocks,
            "use_noise_channel": self.use_noise_channel
        })
        return config




In [ ]:
def create_rwman(input_shape=(None, None, 3), use_noise=False, name="RWMAN"):
    model = RWMAN(
        input_shape=input_shape,
        filters=64,
        num_blocks=128,
        use_noise_channel=use_noise,
        name=name
    )
    return model